Import the necessary libraries.

In [1]:
import sqlite3
import pandas as pd
import requests
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [10]:
date = c.execute("""
            SELECT Date
            FROM Matches
            LIMIT 20;
        """).fetchall()
date

[('2010-04-04',),
 ('2009-11-01',),
 ('2009-10-04',),
 ('2010-02-21',),
 ('2009-12-06',),
 ('2010-04-03',),
 ('2009-08-14',),
 ('2010-03-08',),
 ('2009-09-26',),
 ('2009-11-21',),
 ('2009-12-19',),
 ('2010-04-16',),
 ('2010-03-21',),
 ('2009-09-14',),
 ('2010-05-02',),
 ('2009-10-30',),
 ('2010-01-15',),
 ('2009-10-18',),
 ('2010-02-05',),
 ('2009-08-24',)]

In [4]:
URL = 'https://api.darksky.net/forecast'
API_key = '142337611765433b11ce2f729e5db0e7'
latitude = '52.5200'
longitude = '13.4050'
#time = '2011-05-21T00:00:00Z'
          


In [11]:
values = []
for i in range(len(date)):
    time = date[i][0]
    r = requests.get('https://api.darksky.net/forecast/' 
                 + API_key 
                 + "/" 
                 + latitude 
                 + ',' 
                 + longitude 
                 + ',' 
                 + time + "T00:00:00Z"
                )
    print(r.json())
    item = r.json()['daily']['data'][0]
    item = item.get('icon')
    values.append([time, item])
values

{'code': 403, 'error': 'daily usage limit exceeded'}


KeyError: 'daily'

Gathers all dates From matches table and creates a table with date and weather

In [ ]:
# def get_weather(URL, API_key, date):
#     latitude = '52.5200'
#     longitude = '13.4050'
#     date = c.execute("""
#             SELECT Date
#             FROM Matches
#Where Date LIKE '%date'
#             LIMIT 20;
#         """).fetchall()
#     values = []
#     for i in range(len(date)):
#         time = date[i][0]
#         r = requests.get('https://api.darksky.net/forecast/' 
#                      + API_key 
#                      + "/" 
#                      + latitude 
#                      + ',' 
#                      + longitude 
#                      + ',' 
#                      + time + "T00:00:00Z"
#                     )
#         print(r.json())
#         item = r.json()['daily']['data'][0]
#         item = item.get('icon')
#         values.append([time, item])
#     c.execute('DROP TABLE IF EXISTS Weather')
#     c.execute("""
#         CREATE TABLE Weather(
#         Date varchar(255),
#         Icon varchar(255)
#         );
#         """)
#     for i in range(len(values)):
#         c.execute(f"""
#                 INSERT INTO Weather 
#                 VALUES ('{values[i][0]}' ,'{values[i][1]}');
#                 """)

In [ ]:
c.execute('DROP TABLE IF EXISTS Weather')
c.execute("""
    CREATE TABLE Weather(
    Date varchar(255),
    Icon varchar(255)
    );
    """)
for i in range(len(values)):
    c.execute(f"""
            INSERT INTO Weather 
            VALUES ('{values[i][0]}' ,'{values[i][1]}');
            """)

Create a SQLite query for the number of home goals per team named as Home_Goals.

In [ ]:
c.execute('DROP TABLE IF EXISTS Home_Goals')
c.execute("""
            CREATE TABLE Home_Goals AS 
                SELECT DISTINCT(HomeTeam) AS Team,
                SUM(FTHG) AS Goals,
                COUNT(FTR) AS Wins,
                Icon
                From Matches
                JOIN Weather ON Matches.Date = Weather.Date
                WHERE FTR = 'H'
                GROUP BY HomeTeam, Icon
            ;
            """)
c.execute('SELECT * FROM Home_Goals').fetchall()

Create a SQLite query for the number of away goals per team named as Away_Goals.

In [ ]:
c.execute('DROP TABLE IF EXISTS Away_Goals')
c.execute("""
            CREATE TABLE Away_Goals AS
            SELECT DISTINCT(AwayTeam) AS Team,
            SUM(FTAG) AS Goals,
            COUNT(FTR) AS Wins,
            Icon
            From Matches
            JOIN Weather ON Matches.Date = Weather.Date
            WHERE FTR = 'A'
            GROUP BY HomeTeam, Icon
            ;
            """)
c.execute('SELECT * FROM Away_Goals').fetchall()

In [ ]:
def pipeline(season):
    query = """
            SELECT 
            DISTINCT(HomeTeam) AS Teams,
            Div,
            Home_Goals.Goals + Away_Goals.Goals AS Goals,
            Home_Goals.Wins + Away_Goals.Wins AS Wins,
            ROUND(SUM(Home_Goals.Wins + Away_Goals.Wins) * 100.0 / SUM(SUM(Home_Goals.Wins + Away_Goals.Wins)) OVER(PARTITION BY HomeTeam), 0) AS percent,
            Home_Goals.Icon
            FROM Matches
            JOIN Home_Goals ON Matches.HomeTeam = Home_Goals.Team
            JOIN Away_Goals ON Matches.HomeTeam = Away_Goals.Team
            WHERE Div = 'D2' OR Div = 'E0'
            AND Season = '""" + str(season) + """'
            GROUP BY Teams, Home_Goals.Icon
            ;""" 
    c.execute(query)
    df = pd.DataFrame(c.fetchall())
    df.columns = [x[0] for x in c.description]
    df.groupby('Teams')
    return df

Check our function:

In [ ]:
all = pipeline(2011)
all

In [ ]:
def which_division:
    all = pipeline(season)
    D2 = all[all["Div"] == 'D2']
    D2 = D2.sum()
    E0 = all[all["Div"] == 'E0']
    E0 = E0.sum()
    if D2 > EO:
        print('D2')
    else:
        print('E0')

In [ ]:
def rain(season):
    all = pipline(season)
    rain = all[all['Icon'] == 'rain']
    rain

In [ ]:
atlas_user = 'brian_m'
atlas_key = ds.get_api_key('/Users/bmcgarry/.secret/atlas.txt')
cluster_name = 'chocolate-clusters-so6jw'
collection_name = 'soccer_stats'

ds.insert_to_atlas(atlas_user,
                   atlas_key,
                   cluster_name,
                   collection_name,
                   team_stats)